In [27]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [104]:
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error
import random

In [29]:
subject = 'Bahia - Consumo de Cimento (t)'
split_index = 18 #Referente aos 19 anos de input  

In [30]:
data = pd.read_csv('2003_01_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),...,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - IDH Renda,Bahia - IDH Educacao,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Consumo de Cimento (t)
0,2003,5.999222e+08,2.428732e+09,1.863367e+10,1.124993e+10,3.775011e+10,1.809535e+09,6.162382e+08,5.122550e+09,7.083133e+09,...,0.766362,0.669899,3.692839,0.638715,0.598394,1.317344e+08,8.384593e+06,8.566149,1.216359e+08,1658.094
1,2004,5.851223e+08,2.506090e+09,1.964942e+10,1.175278e+10,4.026419e+10,2.015670e+09,5.396413e+08,5.449927e+09,7.384787e+09,...,0.770951,0.673628,3.775586,0.641511,0.603205,1.336785e+08,8.472829e+06,8.602876,1.223016e+08,1733.557
2,2005,5.590308e+08,2.649361e+09,2.158067e+10,1.256500e+10,4.498692e+10,2.402642e+09,3.968813e+08,6.065174e+09,7.948531e+09,...,0.774974,0.677007,3.824208,0.644092,0.607850,1.354495e+08,8.522905e+06,8.631972,1.228206e+08,1668.001
3,2006,5.264270e+08,2.833246e+09,2.405920e+10,1.373622e+10,5.105978e+10,2.889954e+09,2.186862e+08,6.842013e+09,8.653759e+09,...,0.778443,0.680049,3.940207,0.646471,0.612327,1.370423e+08,8.592312e+06,8.653845,1.232167e+08,1934.967
4,2007,4.920899e+08,3.032448e+09,2.671679e+10,1.531605e+10,5.762426e+10,3.397108e+09,4.320482e+07,7.654167e+09,9.379862e+09,...,0.781367,0.682761,4.044809,0.648659,0.616638,1.385256e+08,8.640798e+06,8.676548,1.235991e+08,2259.804
5,2008,4.607989e+08,3.221668e+09,2.918522e+10,1.735413e+10,6.382184e+10,3.843605e+09,0.000000e+00,8.375355e+09,1.000623e+10,...,0.783758,0.685154,4.100371,0.650668,0.620781,1.396949e+08,8.691699e+06,8.687931,1.237178e+08,2678.337
6,2009,4.373329e+08,3.375608e+09,3.109628e+10,1.990007e+10,6.879400e+10,4.148947e+09,0.000000e+00,8.879299e+09,1.041227e+10,...,0.785624,0.687239,4.185427,0.652511,0.624755,1.406176e+08,8.722094e+06,8.695651,1.237698e+08,2825.491
7,2010,4.264713e+08,3.468971e+09,3.208175e+10,2.300349e+10,7.168223e+10,4.232635e+09,0.000000e+00,9.039721e+09,1.047735e+10,...,0.786978,0.689024,4.338533,0.654198,0.628561,1.439300e+08,8.745548e+06,8.730895,1.244348e+08,3349.109
8,2011,4.329931e+08,3.476458e+09,3.177342e+10,2.671402e+10,7.162801e+10,4.014172e+09,0.000000e+00,8.730341e+09,1.008089e+10,...,0.787829,0.690520,4.536829,0.655741,0.632198,1.469771e+08,8.709535e+06,8.759313,1.249553e+08,3615.893
9,2012,4.616776e+08,3.372772e+09,2.980306e+10,3.108128e+10,6.777282e+10,3.413059e+09,1.276614e+08,7.824880e+09,9.102259e+09,...,0.788133,0.691525,4.749958,0.657154,0.635421,1.498366e+08,8.659814e+06,8.775083,1.254646e+08,3744.322


In [31]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),...,Bahia - Desemprego,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - IDH Renda,Bahia - IDH Educacao,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado
0,5.999222e+08,2.428732e+09,1.863367e+10,1.124993e+10,3.775011e+10,1.809535e+09,6.162382e+08,5.122550e+09,7.083133e+09,1.562192e+12,...,8.348779,0.766362,0.669899,3.692839,0.638715,0.598394,1.317344e+08,8.384593e+06,8.566149,1.216359e+08
1,5.851223e+08,2.506090e+09,1.964942e+10,1.175278e+10,4.026419e+10,2.015670e+09,5.396413e+08,5.449927e+09,7.384787e+09,1.574016e+12,...,8.279180,0.770951,0.673628,3.775586,0.641511,0.603205,1.336785e+08,8.472829e+06,8.602876,1.223016e+08
2,5.590308e+08,2.649361e+09,2.158067e+10,1.256500e+10,4.498692e+10,2.402642e+09,3.968813e+08,6.065174e+09,7.948531e+09,1.606744e+12,...,8.250077,0.774974,0.677007,3.824208,0.644092,0.607850,1.354495e+08,8.522905e+06,8.631972,1.228206e+08
3,5.264270e+08,2.833246e+09,2.405920e+10,1.373622e+10,5.105978e+10,2.889954e+09,2.186862e+08,6.842013e+09,8.653759e+09,1.668478e+12,...,8.164348,0.778443,0.680049,3.940207,0.646471,0.612327,1.370423e+08,8.592312e+06,8.653845,1.232167e+08
4,4.920899e+08,3.032448e+09,2.671679e+10,1.531605e+10,5.762426e+10,3.397108e+09,4.320482e+07,7.654167e+09,9.379862e+09,1.767318e+12,...,8.136082,0.781367,0.682761,4.044809,0.648659,0.616638,1.385256e+08,8.640798e+06,8.676548,1.235991e+08
5,4.607989e+08,3.221668e+09,2.918522e+10,1.735413e+10,6.382184e+10,3.843605e+09,0.000000e+00,8.375355e+09,1.000623e+10,1.911366e+12,...,8.102744,0.783758,0.685154,4.100371,0.650668,0.620781,1.396949e+08,8.691699e+06,8.687931,1.237178e+08
6,4.373329e+08,3.375608e+09,3.109628e+10,1.990007e+10,6.879400e+10,4.148947e+09,0.000000e+00,8.879299e+09,1.041227e+10,2.108725e+12,...,8.129293,0.785624,0.687239,4.185427,0.652511,0.624755,1.406176e+08,8.722094e+06,8.695651,1.237698e+08
7,4.264713e+08,3.468971e+09,3.208175e+10,2.300349e+10,7.168223e+10,4.232635e+09,0.000000e+00,9.039721e+09,1.047735e+10,2.367495e+12,...,8.095385,0.786978,0.689024,4.338533,0.654198,0.628561,1.439300e+08,8.745548e+06,8.730895,1.244348e+08
8,4.329931e+08,3.476458e+09,3.177342e+10,2.671402e+10,7.162801e+10,4.014172e+09,0.000000e+00,8.730341e+09,1.008089e+10,2.695778e+12,...,8.084857,0.787829,0.690520,4.536829,0.655741,0.632198,1.469771e+08,8.709535e+06,8.759313,1.249553e+08
9,4.616776e+08,3.372772e+09,2.980306e+10,3.108128e+10,6.777282e+10,3.413059e+09,1.276614e+08,7.824880e+09,9.102259e+09,3.101676e+12,...,13.555307,0.788133,0.691525,4.749958,0.657154,0.635421,1.498366e+08,8.659814e+06,8.775083,1.254646e+08


In [32]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1733.557
1     1668.001
2     1934.967
3     2259.804
4     2678.337
5     2825.491
6     3349.109
7     3615.893
8     3744.322
9     3744.322
10    3932.905
11    4055.140
12    3732.552
13    3424.968
14    3063.300
15    2990.152
16    3245.585
17    3803.900
18    3862.535
19    3948.984
20         NaN
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [33]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),...,Bahia - Desemprego,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - IDH Renda,Bahia - IDH Educacao,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado
0,1.929157,-1.419344,-0.368703,-1.335423,-1.192306,-0.486476,-0.287330,-0.213191,-0.182599,-1.055607,...,-0.934231,-2.399585,-2.069639,-1.357037,-2.072957,-1.816479,-1.704071,-0.774392,-2.166278,-2.102168
1,1.674167,-1.217480,-0.246178,-1.296989,-1.009056,-0.321568,-0.356568,-0.087570,-0.050617,-1.048258,...,-0.954343,-1.810029,-1.666398,-1.243896,-1.702837,-1.551345,-1.477717,-0.264339,-1.683254,-1.630280
2,1.224632,-0.843622,-0.013222,-1.234909,-0.664820,-0.011991,-0.485612,0.148514,0.196037,-1.027918,...,-0.962753,-1.293099,-1.300859,-1.177414,-1.361089,-1.295375,-1.271514,0.025121,-1.300583,-1.262324
3,0.662896,-0.363781,0.285749,-1.145390,-0.222173,0.377858,-0.646686,0.446604,0.504593,-0.989552,...,-0.987526,-0.847453,-0.971951,-1.018807,-1.046149,-1.048595,-1.086060,0.426329,-1.012906,-0.981560
4,0.071297,0.156028,0.606319,-1.024640,0.256307,0.783581,-0.805308,0.758244,0.822284,-0.928125,...,-0.995694,-0.471752,-0.678605,-0.875782,-0.756449,-0.811029,-0.913362,0.706603,-0.714309,-0.710507
5,-0.467822,0.649791,0.904072,-0.868865,0.708044,1.140778,-0.844361,1.034980,1.096339,-0.838601,...,-1.005327,-0.164654,-0.419748,-0.799811,-0.490425,-0.582705,-0.777211,1.000839,-0.564596,-0.626324
6,-0.872120,1.051493,1.134593,-0.674273,1.070461,1.385051,-0.844361,1.228354,1.273989,-0.715947,...,-0.997656,0.075181,-0.194313,-0.683512,-0.246512,-0.363647,-0.669781,1.176539,-0.463072,-0.589487
7,-1.059257,1.295119,1.253465,-0.437071,1.280982,1.452001,-0.844361,1.289911,1.302467,-0.555126,...,-1.007454,0.249094,-0.001227,-0.474167,-0.023145,-0.153882,-0.284118,1.312113,0.000463,-0.118038
8,-0.946891,1.314657,1.216272,-0.153467,1.277030,1.277231,-0.844361,1.171195,1.129001,-0.351104,...,-1.010496,0.358425,0.160580,-0.203034,0.181244,0.046564,0.070662,1.103937,0.374211,0.250938
9,-0.452682,1.044090,0.978599,0.180333,0.996028,0.796342,-0.728965,0.823751,0.700825,-0.098846,...,0.570292,0.397511,0.269325,0.088382,0.368217,0.224162,0.403592,0.816524,0.581629,0.611962


In [34]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1733.557
1     1668.001
2     1934.967
3     2259.804
4     2678.337
5     2825.491
6     3349.109
7     3615.893
8     3744.322
9     3744.322
10    3932.905
11    4055.140
12    3732.552
13    3424.968
14    3063.300
15    2990.152
16    3245.585
17    3803.900
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [35]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),...,Bahia - Desemprego,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - IDH Renda,Bahia - IDH Educacao,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado
18,0.994288,2.394804,-0.631233,1.044637,2.513792,-0.588445,1.362365,-0.32155,-0.60119,2.228926,...,1.070604,0.517395,0.955188,2.066123,0.759408,1.160231,0.779173,-1.338858,0.436451,0.363642


In [36]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

18    3862.535
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [153]:
def validation_row_splitter(df): 
    pos = df.shape[0] - 2
    return df.drop(pos), df.iloc[pos:pos + 1]

In [154]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(1, split_factor + 1):
        pos = df.shape[0] - (i*2)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [161]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [168]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [169]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[4246962128, 3212046807, 1262751162, 2797361133, 1463072499, 2309084261, 1954551567, 718901426, 1679529324, 2450884342, 128194962, 2810972325, 257787415, 2608994476, 34741384, 491312681, 2784708950, 160564134, 1025005194, 2147948280, 2829257701, 399262429, 3834462203, 3286410179, 1980023505, 905423653, 2108704089, 1356698297, 3993928404, 2297131272, 4252617430, 2805123550, 3974783926, 3573813841, 2261340082, 2760331165, 4108279771, 1672733343, 3937778573, 2132564355, 3887127259, 781545930, 875311847, 2928258264, 1505466544, 1569780544, 4177815517, 2928789410, 3429314719, 2020385722, 1807914858, 953328186, 1350581768, 1860132138, 908223643, 3256389990, 393294798, 1132373250, 3490798274, 1638089477, 4156015832, 2962519410, 482045136, 347676509, 2235296034, 1941871571, 4069333577, 3224195195, 2402116018, 1314677188, 3858966778, 2750266443, 489028578, 426289086, 1687750052, 1832485876, 596116340, 831317958, 3007412922, 719399375, 2402062732, 507440790, 1581769709, 984610373, 4125183177, 41

2023-10-07 10:18:04.252336: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 10092.1630859375


Step: 20 ___________________________________________
val_loss: 90701.5546875


Step: 21 ___________________________________________
val_loss: 14890.796875


Step: 22 ___________________________________________
val_loss: 23254.744140625


Step: 23 ___________________________________________
val_loss: 6804.31396484375


Step: 24 ___________________________________________
val_loss: 23109.953125


Step: 25 ___________________________________________
val_loss: 9516.5390625


Step: 26 ___________________________________________
val_loss: 137074.703125


Step: 27 ___________________________________________
val_loss: 77961.9765625


Step: 28 ___________________________________________
val_loss: 6316.46826171875


Step: 29 ___________________________________________
val_loss: 16287.0234375


Step: 30 ___________________________________________
val_loss: 79806.9921875


Step: 31 ___________________________________________
val_loss: 20746.7734375


Step: 32 _________

2023-10-07 10:23:38.333025: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-07 10:23:38.370553: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-07 10:23:38.400448: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 68356.8828125


Step: 39 ___________________________________________
val_loss: 59201.89453125


Step: 40 ___________________________________________
val_loss: 20194.3828125


Step: 41 ___________________________________________
val_loss: 5614.56787109375


Step: 42 ___________________________________________
val_loss: 5673.35498046875


Step: 43 ___________________________________________
val_loss: 79806.9921875


Step: 44 ___________________________________________
val_loss: 6052.85546875


Step: 45 ___________________________________________
val_loss: 1602.3670654296875
winner_seed: 1569780544


Step: 46 ___________________________________________
val_loss: 79807.0234375


Step: 47 ___________________________________________
val_loss: 26103.365234375


Step: 48 ___________________________________________
val_loss: 6789.03125


Step: 49 ___________________________________________
val_loss: 79806.984375


Step: 50 ___________________________________________
val_loss: 79806.99

2023-10-07 10:28:21.949015: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 11009.4365234375


Step: 55 ___________________________________________
val_loss: 79807.1796875


Step: 56 ___________________________________________
val_loss: 79814.0390625


Step: 57 ___________________________________________
val_loss: 79806.9921875


Step: 58 ___________________________________________
val_loss: 18957.830078125


Step: 59 ___________________________________________
val_loss: 74671.6171875


Step: 60 ___________________________________________
val_loss: 79806.9921875


Step: 61 ___________________________________________
val_loss: 18464.375


Step: 62 ___________________________________________
val_loss: 50918.703125


Step: 63 ___________________________________________
val_loss: 2882.357177734375


Step: 64 ___________________________________________
val_loss: 15009.3232421875


Step: 65 ___________________________________________
val_loss: 9477.767578125


Step: 66 ___________________________________________
val_loss: 79807.0234375


Step: 67 _________

In [170]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 460ms/step - loss: 9945411.0000 - val_loss: 11112891.0000
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 9700178.0000 - val_loss: 20008458.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 7437817.0000 - val_loss: 24483320.0000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 8642278.0000 - val_loss: 21302044.0000
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 7672139.5000 - val_loss: 21589426.0000
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 6820535.5000 - val_loss: 22248202.0000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 6438931.0000 - val_loss: 22779206.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 6330566.5000 - val_loss: 22850710.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 6599383.0000 - val_lo

1/1 [==============================] - 0s 13ms/step - loss: 2195196.2500 - val_loss: 6337697.5000
Epoch 74/10000
1/1 [==============================] - 0s 13ms/step - loss: 3385215.5000 - val_loss: 7350626.5000
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 3029016.5000 - val_loss: 4152600.7500
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 2293681.0000 - val_loss: 5759989.5000
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 2357885.7500 - val_loss: 6417905.5000
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 1994032.1250 - val_loss: 5783406.0000
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 1951350.6250 - val_loss: 5897053.5000
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 2615079.5000 - val_loss: 4386808.5000
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 1814731.1250 - val_loss: 5544751.500

1/1 [==============================] - 0s 14ms/step - loss: 1436326.0000 - val_loss: 3113669.0000
Epoch 146/10000
1/1 [==============================] - 0s 14ms/step - loss: 1072802.1250 - val_loss: 3485668.7500
Epoch 147/10000
1/1 [==============================] - 0s 15ms/step - loss: 953172.1250 - val_loss: 3644150.0000
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 961987.0625 - val_loss: 2961838.0000
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 1200633.2500 - val_loss: 3111525.2500
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 998002.3125 - val_loss: 2788137.7500
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 1194313.3750 - val_loss: 2515923.5000
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 1356682.5000 - val_loss: 2353399.5000
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 1001833.1875 - val_loss: 268852

1/1 [==============================] - 0s 13ms/step - loss: 589349.8750 - val_loss: 944877.4375
Epoch 218/10000
1/1 [==============================] - 0s 14ms/step - loss: 497127.9688 - val_loss: 796403.7500
Epoch 219/10000
1/1 [==============================] - 0s 14ms/step - loss: 696116.3750 - val_loss: 833538.1875
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 545795.6250 - val_loss: 798670.8125
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 807709.3125 - val_loss: 974391.4375
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 521669.3438 - val_loss: 974478.3125
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 558545.3750 - val_loss: 1091351.1250
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 448433.3438 - val_loss: 846874.5625
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 632073.3125 - val_loss: 941442.5000
Epoch 2

1/1 [==============================] - 0s 14ms/step - loss: 347523.7188 - val_loss: 630366.1250
Epoch 291/10000
1/1 [==============================] - 0s 14ms/step - loss: 524690.1250 - val_loss: 699876.4375
Epoch 292/10000
1/1 [==============================] - 0s 14ms/step - loss: 406097.5312 - val_loss: 470644.5938
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 294692.4062 - val_loss: 521543.6250
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 327356.2812 - val_loss: 416822.0938
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 344350.9062 - val_loss: 489733.6562
Epoch 296/10000
1/1 [==============================] - 0s 15ms/step - loss: 336158.2812 - val_loss: 458895.5000
Epoch 297/10000
1/1 [==============================] - 0s 15ms/step - loss: 343941.6875 - val_loss: 379568.8438
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 424960.1250 - val_loss: 654269.2500
Epoch 29

...

Epoch 732/10000
1/1 [==============================] - 0s 14ms/step - loss: 210643.5625 - val_loss: 10400.3809
Epoch 733/10000
1/1 [==============================] - 0s 14ms/step - loss: 242983.1719 - val_loss: 20923.5078
Epoch 734/10000
1/1 [==============================] - 0s 13ms/step - loss: 246888.8281 - val_loss: 7735.9468
Epoch 735/10000
1/1 [==============================] - 0s 14ms/step - loss: 166383.0156 - val_loss: 31196.3809
Epoch 736/10000
1/1 [==============================] - 0s 14ms/step - loss: 206460.9062 - val_loss: 7517.8066
Epoch 737/10000
1/1 [==============================] - 0s 13ms/step - loss: 133150.5312 - val_loss: 7335.9766
Epoch 738/10000
1/1 [==============================] - 0s 13ms/step - loss: 128643.3516 - val_loss: 34487.0273
Epoch 739/10000
1/1 [==============================] - 0s 13ms/step - loss: 180062.0312 - val_loss: 6863.9292
Epoch 740/10000
1/1 [==============================] - 0s 14ms/step - loss: 113844.6094 - val_loss: 37809.5273
Epoch

Epoch 806/10000
1/1 [==============================] - 0s 13ms/step - loss: 200112.7188 - val_loss: 67129.4922
Epoch 807/10000
1/1 [==============================] - 0s 13ms/step - loss: 221270.7344 - val_loss: 9615.1992
Epoch 808/10000
1/1 [==============================] - 0s 13ms/step - loss: 113157.6016 - val_loss: 20835.6230
Epoch 809/10000
1/1 [==============================] - 0s 13ms/step - loss: 161013.9062 - val_loss: 9550.4990
Epoch 810/10000
1/1 [==============================] - 0s 13ms/step - loss: 195312.5000 - val_loss: 8985.0459
Epoch 811/10000
1/1 [==============================] - 0s 13ms/step - loss: 135632.3906 - val_loss: 16744.3750
Epoch 812/10000
1/1 [==============================] - 0s 13ms/step - loss: 225891.8125 - val_loss: 9949.0205
Epoch 813/10000
1/1 [==============================] - 0s 13ms/step - loss: 207790.4219 - val_loss: 8942.8359
Epoch 814/10000
1/1 [==============================] - 0s 15ms/step - loss: 157998.4219 - val_loss: 19944.1855
Epoch 

1/1 [==============================] - 0s 13ms/step - loss: 195113.1406 - val_loss: 102439.6484
Epoch 880/10000
1/1 [==============================] - 0s 13ms/step - loss: 172650.2344 - val_loss: 66995.3047
Epoch 881/10000
1/1 [==============================] - 0s 13ms/step - loss: 123852.1172 - val_loss: 77464.6250
Epoch 882/10000
1/1 [==============================] - 0s 13ms/step - loss: 108530.1797 - val_loss: 69787.5391
Epoch 883/10000
1/1 [==============================] - 0s 13ms/step - loss: 78927.9609 - val_loss: 62616.1094
Epoch 884/10000
1/1 [==============================] - 0s 14ms/step - loss: 113432.2031 - val_loss: 65336.8867
Epoch 885/10000
1/1 [==============================] - 0s 13ms/step - loss: 128131.7031 - val_loss: 63581.5469
Epoch 886/10000
1/1 [==============================] - 0s 13ms/step - loss: 139549.9531 - val_loss: 62532.6680
Epoch 887/10000
1/1 [==============================] - 0s 13ms/step - loss: 158177.5625 - val_loss: 79776.8516
Epoch 888/10000
1

Epoch 953/10000
1/1 [==============================] - 0s 14ms/step - loss: 98067.2578 - val_loss: 13034.3564
Epoch 954/10000
1/1 [==============================] - 0s 13ms/step - loss: 115738.5938 - val_loss: 14024.5537
Epoch 955/10000
1/1 [==============================] - 0s 14ms/step - loss: 139219.0156 - val_loss: 13353.5391
Epoch 956/10000
1/1 [==============================] - 0s 13ms/step - loss: 164584.7969 - val_loss: 21390.8828
Epoch 957/10000
1/1 [==============================] - 0s 14ms/step - loss: 188138.5156 - val_loss: 31147.5137
Epoch 958/10000
1/1 [==============================] - 0s 14ms/step - loss: 131021.3828 - val_loss: 24557.2969
Epoch 959/10000
1/1 [==============================] - 0s 13ms/step - loss: 147439.5000 - val_loss: 19559.4668
Epoch 960/10000
1/1 [==============================] - 0s 13ms/step - loss: 149904.3594 - val_loss: 30439.4375
Epoch 961/10000
1/1 [==============================] - 0s 13ms/step - loss: 59796.6836 - val_loss: 20853.9766
Epo

1/1 [==============================] - 0s 14ms/step - loss: 115159.7812 - val_loss: 55426.2617
Epoch 1027/10000
1/1 [==============================] - 0s 13ms/step - loss: 153175.8281 - val_loss: 72842.9531
Epoch 1028/10000
1/1 [==============================] - 0s 14ms/step - loss: 130676.8828 - val_loss: 50934.8711
Epoch 1029/10000
1/1 [==============================] - 0s 14ms/step - loss: 231952.5469 - val_loss: 123059.7734
Epoch 1030/10000
1/1 [==============================] - 0s 14ms/step - loss: 185807.3125 - val_loss: 61241.1562
Epoch 1031/10000
1/1 [==============================] - 0s 13ms/step - loss: 180800.4375 - val_loss: 65033.4805
Epoch 1032/10000
1/1 [==============================] - 0s 14ms/step - loss: 197297.9219 - val_loss: 52961.1367
Epoch 1033/10000
1/1 [==============================] - 0s 14ms/step - loss: 176079.9375 - val_loss: 64090.7305
Epoch 1034/10000
1/1 [==============================] - 0s 13ms/step - loss: 86186.6484 - val_loss: 69290.3672
Epoch 103

In [171]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 34ms/step
[3862.535](test_target) - [[3216.1357]](prediction) = 646.3992578124999


In [172]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [173]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 10, winner_seed)
predictions

train_input:


,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),...,Bahia - Desemprego,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - IDH Renda,Bahia - IDH Educacao,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado
0,1.527752,-1.467854,-1.491268,-1.100590,-1.473178,-1.554258,1.737826,-1.544962,-1.575459,-0.947154,...,1.949741,-1.866952,-1.760277,-1.370437,-1.699938,-1.619152,-1.510012,-1.897934,-1.761678,-1.803303
1,1.296273,-1.268797,-1.288015,-1.001649,-1.274601,-1.322996,1.416773,-1.317990,-1.333517,-0.915679,...,1.160922,-1.219176,-1.204024,-1.051825,-1.191545,-1.178694,-1.084152,-1.152153,-1.121674,-1.116417
2,0.888189,-0.900137,-0.901571,-0.841837,-0.901572,-0.888853,0.818398,-0.891436,-0.881367,-0.828561,...,0.831076,-0.651197,-0.699779,-0.864609,-0.722124,-0.753458,-0.696206,-0.728914,-0.614639,-0.580816
3,0.378248,-0.426967,-0.405617,-0.611390,-0.421901,-0.342139,0.071499,-0.352848,-0.315739,-0.664233,...,-0.140558,-0.161542,-0.246065,-0.417964,-0.289525,-0.343488,-0.347297,-0.142282,-0.233468,-0.172132
4,-0.158802,0.085614,0.126167,-0.300543,0.096600,0.226836,-0.664026,0.210222,0.266632,-0.401132,...,-0.460910,0.251261,0.158594,-0.015200,0.108403,0.051173,-0.022386,0.267523,0.162170,0.222415
5,-0.648211,0.572512,0.620101,0.100467,0.586121,0.727759,-0.845118,0.710226,0.769014,-0.017691,...,-0.838756,0.588686,0.515675,0.198737,0.473811,0.430482,0.233766,0.697744,0.360539,0.344954
6,-1.015230,0.968628,1.002505,0.601403,0.978852,1.070322,-0.845118,1.059614,1.094672,0.507655,...,-0.537864,0.852206,0.826654,0.526241,0.808847,0.794396,0.435883,0.954646,0.495058,0.398574
7,-1.185112,1.208867,1.199698,1.212030,1.206981,1.164211,-0.845118,1.170835,1.146875,1.196471,...,-0.922168,1.043293,1.093008,1.115766,1.115663,1.142873,1.161464,1.152879,1.109239,1.084820
8,-1.083107,1.228133,1.138001,1.942110,1.202698,0.919118,-0.845118,0.956340,0.828889,2.070323,...,-1.041482,1.163421,1.316213,1.879290,1.396408,1.475868,1.828940,0.848491,1.604452,1.621906


train_target:


,Bahia - Consumo de Cimento (t)
0,1733.557
1,1668.001
2,1934.967
3,2259.804
4,2678.337
5,2825.491
6,3349.109
7,3615.893
8,3744.322


test_input:


...

,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),Contratação Jurídico SFH (Bi R$),Direcionamento Aquisição (R$),...,Bahia - Desemprego,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - IDH Renda,Bahia - IDH Educacao,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado
18,0.994288,2.394804,-0.631233,1.044637,2.513792,-0.588445,1.362365,-0.32155,-0.60119,2.228926,...,1.070604,0.517395,0.955188,2.066123,0.759408,1.160231,0.779173,-1.338858,0.436451,0.363642


test_target:


,Bahia - Consumo de Cimento (t)
18,3862.535


1/1 [==============================] - 0s 37ms/step
Error: 646.3992578124999




[3356.318359375,
 3513.29931640625,
 3024.245849609375,
 3512.199951171875,
 3784.771728515625,
 3385.305419921875,
 3132.135986328125,
 3743.965087890625,
 3265.08935546875,
 3216.1357421875]

In [174]:
display(targets)
display(predictions)

[3744.322,
 3932.905,
 4055.14,
 3732.551999999999,
 3424.967999999999,
 3063.3,
 2990.152,
 3245.585,
 3803.9,
 3862.535]

[3356.318359375,
 3513.29931640625,
 3024.245849609375,
 3512.199951171875,
 3784.771728515625,
 3385.305419921875,
 3132.135986328125,
 3743.965087890625,
 3265.08935546875,
 3216.1357421875]

In [175]:
mae = mean_absolute_error(predictions, targets)
mae

456.62386484375

In [176]:
porcentage = mae/np.mean(targets)
porcentage

0.12735163656951531